In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib

In [4]:
postulantes = pd.read_csv("postulantes_amp.csv")

In [5]:
avisos = pd.read_csv("avisos_amp.csv")

In [6]:
len(avisos)

25288

In [7]:
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [8]:
no_postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv")
no_postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112366050,1Qdlz6L,0
1,1112112513,0zk5b9a,0
2,1112440465,55ZxxX,0
3,1112311267,mzbNQRY,0
4,1112468890,pzNM0zM,0


In [9]:
postulaciones=postulaciones.append(no_postulaciones, ignore_index=True)
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)

In [10]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [11]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [12]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [13]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [14]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [15]:
vistas['cantidad'].value_counts()

1.000000    118603
0.111111     79690
0.142857     79070
0.200000     78950
0.125000     78640
0.166667     78573
0.100000     77604
0.090909     76879
0.333333     75838
0.250000     75275
0.076923     75262
0.083333     75066
0.071429     74647
0.066667     72883
0.062500     71762
0.058824     70607
0.500000     69620
0.055556     69389
0.052632     67594
0.050000     66264
0.045455     64829
0.047619     64482
0.041667     61959
0.043478     61745
0.040000     60253
0.037037     58859
0.038462     58116
0.035714     57955
0.034483     56494
0.033333     55040
             ...  
0.009336         1
0.007819         1
0.042088         1
0.017405         1
0.119883         1
0.016145         1
0.036918         1
0.084886         1
0.030995         1
0.030032         1
0.050167         1
0.078775         1
0.079422         1
0.218310         1
0.567010         1
0.068515         1
0.135667         1
0.011403         1
0.031079         1
0.022704         1
0.092751         1
0.901587    

In [16]:
vistas['vio_anuncio'] = 1

In [17]:
postulaciones = postulaciones.sample(n=1000000)

In [18]:
postulaciones.sepostulo.sum()

500676

In [19]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')

In [20]:
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')

In [21]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')

In [22]:
def vio_anuncio(x):
    return 1 if x > 0 else 0

In [23]:
postulaciones['cantidad'] = postulaciones['cantidad'].fillna(0)

In [24]:
postulaciones['vio_anuncio'] = postulaciones['cantidad'].apply(lambda x: vio_anuncio(x))
postulaciones.head()

,idaviso,idpostulante,sepostulo,Unnamed: 0_x,edad,Doctorado,Master,Posgrado,Secundario,Terciario/Técnico,...,conta,prog,tecno,telemrkt,teso,almacen,rrhh,limpieza,cantidad,vio_anuncio
0,1112447011,zvajoZ5,0,180727,24.0,0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0
1,1112447011,bO4P4KE,0,67674,56.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0
2,1112447011,6vNmmx,1,9233,35.0,0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.003521,1
3,1112447011,aMLOb5,0,33586,32.0,0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0
4,1112447011,wVYlxAV,1,71013,23.0,0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0


In [25]:
postulaciones['vio_anuncio'].sum()

134763

In [26]:
postulaciones.drop(['Unnamed: 0_x','denominacion_empresa','Unnamed: 0_y','edad'], axis=1, inplace=True)
postulaciones.head()

,idaviso,idpostulante,sepostulo,Doctorado,Master,Posgrado,Secundario,Terciario/Técnico,Universitario,masculino,...,conta,prog,tecno,telemrkt,teso,almacen,rrhh,limpieza,cantidad,vio_anuncio
0,1112447011,zvajoZ5,0,0,0.0,0.0,1.0,0.0,0,1,...,0,0,0,0,0,0,0,0,0.000000,0
1,1112447011,bO4P4KE,0,0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0.000000,0
2,1112447011,6vNmmx,1,0,0.0,0.0,1.0,0.0,0,1,...,0,0,0,0,0,0,0,0,0.003521,1
3,1112447011,aMLOb5,0,0,0.0,0.0,1.0,0.0,0,1,...,0,0,0,0,0,0,0,0,0.000000,0
4,1112447011,wVYlxAV,1,0,0.0,0.0,1.0,0.0,0,1,...,0,0,0,0,0,0,0,0,0.000000,0


In [27]:
#features = ['idaviso', 'idpostulante', 'sepostulo', 'edad', 'Doctorado', 'Master', 'Posgrado', 'Secundario', 'Terciario/Técnico', 'Universitario', 'nombre_area', 'pide_hombre', 'pide_mujer', 'ingles', 'experiencia', 'secundario', 'graduados', 'estudiantes', 'pide_vendedor', 'pide_administrativo', 'pide_tecnico', 'pide_analista', 'pide_comercial', 'pide_asistente', 'pide_ejecutivo', 'pide_ingeniero', 'pide_operario', 'pide_desarollador', 'cantidad', 'masculino', 'femenino', 'gran_BA', 'capital_f', 'fueraOeste_GBA', 'cordoba', 'full_time', 'part_time', 'teletrabajo', 'pasantia', 'senior', 'junior', 'jefe', 'gerente', 'ventas', 'admin', 'producc', 'comercial', 'at_cliente', 'conta', 'prog', 'tecno', 'telemrkt', 'teso', 'almacen', 'rrhh', 'limpieza']
features=list(postulaciones.columns)
features

['idaviso',
 'idpostulante',
 'sepostulo',
 'Doctorado',
 'Master',
 'Posgrado',
 'Secundario',
 'Terciario/Técnico',
 'Universitario',
 'masculino',
 'femenino',
 'pide_hombre',
 'pide_mujer',
 'ingles',
 'experiencia',
 'secundario',
 'graduados',
 'estudiantes',
 'pide_vendedor',
 'pide_administrativo',
 'pide_tecnico',
 'pide_analista',
 'pide_comercial',
 'pide_asistente',
 'pide_ejecutivo',
 'pide_ingeniero',
 'pide_operario',
 'pide_desarollador',
 'gran_BA',
 'capital_f',
 'fueraOeste_GBA',
 'cordoba',
 'full_time',
 'part_time',
 'teletrabajo',
 'pasantia',
 'senior',
 'junior',
 'jefe',
 'gerente',
 'ventas',
 'admin',
 'producc',
 'comercial',
 'at_cliente',
 'conta',
 'prog',
 'tecno',
 'telemrkt',
 'teso',
 'almacen',
 'rrhh',
 'limpieza',
 'cantidad',
 'vio_anuncio']

In [28]:
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')

In [29]:
features

['Doctorado',
 'Master',
 'Posgrado',
 'Secundario',
 'Terciario/Técnico',
 'Universitario',
 'masculino',
 'femenino',
 'pide_hombre',
 'pide_mujer',
 'ingles',
 'experiencia',
 'secundario',
 'graduados',
 'estudiantes',
 'pide_vendedor',
 'pide_administrativo',
 'pide_tecnico',
 'pide_analista',
 'pide_comercial',
 'pide_asistente',
 'pide_ejecutivo',
 'pide_ingeniero',
 'pide_operario',
 'pide_desarollador',
 'gran_BA',
 'capital_f',
 'fueraOeste_GBA',
 'cordoba',
 'full_time',
 'part_time',
 'teletrabajo',
 'pasantia',
 'senior',
 'junior',
 'jefe',
 'gerente',
 'ventas',
 'admin',
 'producc',
 'comercial',
 'at_cliente',
 'conta',
 'prog',
 'tecno',
 'telemrkt',
 'teso',
 'almacen',
 'rrhh',
 'limpieza',
 'cantidad',
 'vio_anuncio']

In [30]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))

Train:  920103 Test:  48427


In [31]:
x_train=train[features]
y_train=train['sepostulo']

In [32]:
x_test=test[features]
y_test=test['sepostulo']

In [39]:
#ejecutar esta celda para gridsearch.
for x in range(8,13):
    if x != 10:
        knn = KNeighborsClassifier(n_neighbors=x, weights='uniform', n_jobs=-1)
        knn.fit(x_train, y_train)
        joblib.dump(knn, "../../Data/fiuba_entrenamiento/gian/modelos/KNN-"+str(x)+"vecinos.pkl")
        print(knn.score(x_test, y_test)*100)

67.21758532352699
67.61631270142964
67.80224774812
68.20097512602264


In [ ]:
knn = KNeighborsClassifier(n_neighbors=20, weights='distance', n_jobs=-1)
knn.fit(x_train, y_train)

In [ ]:
datetime.now()

In [ ]:
joblib.dump(knn, "../../Data/fiuba_entrenamiento/gian/modelos/KNN-7(distance)-20neigborghs.pkl")

In [27]:
#knn = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/KNN-5(distance).pkl")

In [ ]:
score=knn.score(x_test,y_test)*100
print(score)

In [69]:
vistas.head()

,idaviso,idpostulante,cantidad,vio_anuncio
0,18,BolNL,0.000000,0
1,48375,RwVdKR,0.006024,1
2,169730,1KjXB,0.000000,0
3,169730,2AKzxa,0.000000,0
4,169730,6LJ64,0.000000,0


In [35]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idpostulante','idaviso'],how='left')
print(len(prediccion))
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000
100000


,id,idaviso,idpostulante,Unnamed: 0_x,edad,Doctorado,Master,Posgrado,Secundario,Terciario/Técnico,...,conta,prog,tecno,telemrkt,teso,almacen,rrhh,limpieza,cantidad,vio_anuncio
33477,8023,1111753681,Eze8pXo,348752,26.0,0,0.00,0.0,1.0,0.0,...,0,0,0,0,1,0,0,0,0.036697,1.0
81603,29324,1112370839,6BZzPj,209971,37.0,0,0.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,NaN
18969,77228,1112464306,akjmdJ0,187710,26.0,0,0.00,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,NaN
32636,43909,1112425681,6VVXZR,228084,29.0,0,0.00,0.0,1.0,0.0,...,0,0,0,1,0,0,0,0,0.062500,1.0
91916,90838,1112469033,EXVeVo,226156,43.0,0,0.00,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0.000000,NaN
87132,39766,1112417896,5LJvdp,4675,43.0,0,0.00,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,NaN
71063,37630,1112414169,A3DWZxm,70250,29.0,0,0.00,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.022727,1.0
51550,61414,1112455493,YjA4Kpw,103244,30.0,0,0.00,0.0,1.0,0.0,...,0,0,0,0,0,0,0,1,0.022727,1.0
87819,42298,1112423310,8Q3rLz,232267,42.0,0,0.15,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,NaN
27145,49347,1112437827,bO4XwqY,282159,24.0,0,0.00,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0.000000,NaN


In [36]:
prediccion['vio_anuncio'] = prediccion['vio_anuncio'].fillna(0)
prediccion.sample(n=10)

,id,idaviso,idpostulante,Unnamed: 0_x,edad,Doctorado,Master,Posgrado,Secundario,Terciario/Técnico,...,conta,prog,tecno,telemrkt,teso,almacen,rrhh,limpieza,cantidad,vio_anuncio
7852,89707,1112467034,2zQAdLY,100159,28.0,0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.002841,1.0
14023,61434,1112455501,A3XzpX6,168284,22.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.016667,1.0
15574,66547,1112459934,EKdW2N,232170,45.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.111111,1.0
56367,5919,1111605016,ZRrvJP,10973,33.0,0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0.0
93114,92265,1112474536,albk80,8279,33.0,0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0.0
74724,20778,1112263302,aZEGaZ,11345,33.0,0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0.0
50120,3425,1111378526,8jAxz,202628,40.0,0,0.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0.000000,0.0
38553,36047,1112411398,KBrYd9m,392110,36.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.095238,1.0
45694,1914,1111158052,1lRJGz,205757,200.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0.0
64697,11626,1111962901,alWalJ,8893,33.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0.0


In [37]:
prediccion['vio_anuncio'].sum()

34161.0

In [38]:
x_final=prediccion[features]
y_final=knn.predict_proba(x_final)
y_final

array([[1.        , 0.        ],
       [0.88056649, 0.11943351],
       [1.        , 0.        ],
       ...,
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.93333333, 0.06666667]])

In [39]:
y_final2=[]
print(knn.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [40]:
prediccion['sepostulo']=y_final2

In [41]:
prediccion.head()

,id,idaviso,idpostulante,Unnamed: 0_x,edad,Doctorado,Master,Posgrado,Secundario,Terciario/Técnico,...,prog,tecno,telemrkt,teso,almacen,rrhh,limpieza,cantidad,vio_anuncio,sepostulo
0,0,739260,6M9ZQR,208743,42.0,0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0.0,0.0,0.000000
1,1,739260,6v1xdL,8762,30.0,0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0.119434
2,2,739260,ezRKm9,227418,36.0,0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0.0,0.0,0.000000
3,3,758580,1Q35ej,210702,68.0,0,0.0,1.0,1.0,0.0,...,0,1,0,0,0,0,0,0.0,0.0,0.000000
4,4,758580,EAN4J6,220121,32.0,0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0.0,0.0,0.000000


In [42]:
prediccion.drop(features, axis=1, inplace=True )

In [43]:
prediccion.drop(['idaviso','idpostulante','Unnamed: 0_x', 'Unnamed: 0_y','edad','denominacion_empresa'], axis=1, inplace=True)

In [44]:
prediccion.head()

,id,sepostulo
0,0,0.000000
1,1,0.119434
2,2,0.000000
3,3,0.000000
4,4,0.000000


In [45]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion112.csv",index=False)

In [46]:
prediccion.sepostulo.mean()

0.5774263288723773